In [93]:
from __future__ import division
from __future__ import print_function

import argparse
import time

import numpy as np
import scipy.sparse as sp
import torch
import torchvision
from torch import optim
import pandas as pd


from gae.model import GCNModelVAE

from gae.optimizer import loss_function
from gae.utils import load_data, mask_test_edges, preprocess_graph, get_roc_score

import import_ipynb
from gaemain import gae_for

from scipy.stats.stats import pearsonr  
from scipy import spatial

In [94]:
# N=7
# N2=(N-1)*10 + N
# image_path="./orl_dataset/person"+str(N)+"/train_images/"+str(N2)+"_"+str(N)+".jpg"
# Z=gae_for(image_path)
# Z=Z.detach().numpy()
# # print(N,N2)

In [95]:
testSets=[]
for person in range(1,5):
    path="./orl_dataset/person"+str(person)+"/test_image/"
    files=os.listdir(path)
    count=0
    for image in files:
        image_path=path+image
        # print(image_path)
        count=count+1
        Z=gae_for(image_path)
        # Z=Z.detach().numpy() #convert to Numpy array
        Z = pd.DataFrame(Z) #convert to a dataframe
        testSets.append(Z)
        csv_path="./orl_dataset/person"+str(person)+"/latent_representation_2/testfile"+str(count)+".csv"
        Z.to_csv(csv_path,index=False) #save to file
print(len(testSets))

12


In [96]:
# import glob
# testSets=[]
# ind=0
# for person in range(1,10):
#     tests=glob.glob("./orl_dataset/person"+str(person)+"/latent_representation_2/*.csv")
#     for test in tests: #for eacg file
#         testSets.append(np.loadtxt(test,delimiter=','))  #sign[ind]= content of file, seperated by comma
#         testSets[ind]=testSets[ind][1:]
#         ind+=1
#     print(len(testSets))
# print(tests)
# print(len(testSets))

In [103]:
import numpy as np
import glob
import os

acc=[]
avg_p=[]
avg_recall=[]
avg_fScore=[]
betaSquare=0.5*0.5
r_rate=0 #recognition rate
pr_rate=0 #recognition rate
acc=[]

# files = glob.glob('./orl_dataset/latent1/*.csv')
sign=[]  #15*null

index=0
count=1

for testSet in testSets:
  data = []
  test=[]
  # for file in files:
  TP=0
  TN=0
  FP=0
  FN=0
  tdata=testSet

  # print("Z=",type(Z))
  dist=[]
  pearson_dist=[]
  cosine_dist=[]
  c_dist=[]

  orig_path="./orl_dataset/"
  files=os.listdir(orig_path)
  tests=[]
  # count=0
  for file in files:
    # path=file + '/latent_representation/'
    tests=glob.glob(orig_path + file + '/latent_representation/*.csv')

    sign=[None]*15  #15*null
    ind=0
    # print("files=",len(files))
    for test in tests: #for eacg file
      sign[ind]=np.loadtxt(test,delimiter=',')  #sign[ind]= content of file, seperated by comma
      sign[ind]=sign[ind][1:]
      ind+=1

    # print("test ",tests)
    # print("sign[0]=",sign[0])

    for ele in sign:
      if ele is None:
        break
      dist.append(np.linalg.norm(ele-tdata))
      cosine_dist.append(1 - spatial.distance.cosine(np.array(tdata).flatten() ,np.array(ele).flatten()))
      pearson_dist.append(pearsonr(np.array(tdata).flatten() ,np.array(ele).flatten())[0])
      c_dist.append(pearsonr(np.array(tdata).flatten() ,np.array(ele).flatten())[0])

  # min_ind=dist.index(min(dist))
  # print("len cosine_dist",len(cosine_dist))
  # print("pearson_dist",pearson_dist)
  max_ind=pearson_dist.index(max(pearson_dist))
  max_cosine=cosine_dist.index(max(cosine_dist))
  max_c_dist=c_dist.index(max(c_dist))
  print("pearson=",max_ind)
  print("cosine=",max_cosine)
  print("c_dist=",max_c_dist)
  print("matched with=", max_c_dist//10 + 1)
  print("index=", index//3 + 1)
  if(index//3 + 1 != max_c_dist//10 + 1):
    print("\t\t\t\tthis is wrong")
    TP=0
    TN=13
    FP=1
    FP=1
  elif(index//3 +1 == max_c_dist//10 + 1):
    r_rate+=1
    TP=1
    TN=14
    FN=0
    FP=0
  #Measures

  Accuracy = (TP + TN )/ (TP + TN + FP + FN) #(all correct / all)
  Misclassification = (FP + FN )/ (TP + TN + FP + FN) #(all incorrect / all)
  Precision  = TP / (TP + FP) #(true positives / predicted positives)
  deno=TP+FP
  Sensitivity   = TP / (deno) #aka Recall (true positives / all actual positives)
  Specificity  =TN / (TN + FP) #(true negatives / all actual negatives)
  F1_deno=(Precision+Sensitivity)
  if(F1_deno==0):
    F1_Score=0
  else:
    F1_Score=(1+betaSquare)*((Precision*Sensitivity)/(betaSquare*F1_deno))
  # print("\nAccuracy: {}\nMisclassification: {}\nPrecision: {}\nSensitivity: {}\nSpecificity: {}\nF1_Score: {}\n".format(Accuracy,Misclassification,Precision,Sensitivity,Specificity,F1_Score))
  acc.append(Accuracy)
  avg_p.append(Precision)
  avg_recall.append(Sensitivity)
  avg_fScore.append(F1_Score)
  index=index+1
print(r_rate)
print("Total Test Case: {}\nAverage Accuracy: {}\nRecognition Rate: {}\nAverage Precision: {}\nAverage Recall: {}\nAverage F0.5 Score: {}".format(len(acc),(sum(acc)/len(acc)),r_rate/len(acc),(sum(avg_p)/len(avg_p)),(sum(avg_recall)/len(avg_recall)),(sum(avg_fScore)/len(avg_fScore))))

pearson= 34
cosine= 34
c_dist= 34
matched with= 4
index= 1
				this is wrong
pearson= 85
cosine= 85
c_dist= 85
matched with= 9
index= 1
				this is wrong
pearson= 51
cosine= 51
c_dist= 51
matched with= 6
index= 1
				this is wrong
pearson= 73
cosine= 73
c_dist= 73
matched with= 8
index= 2
				this is wrong
pearson= 11
cosine= 11
c_dist= 11
matched with= 2
index= 2
pearson= 54
cosine= 54
c_dist= 54
matched with= 6
index= 2
				this is wrong
pearson= 6
cosine= 6
c_dist= 6
matched with= 1
index= 3
				this is wrong
pearson= 56
cosine= 56
c_dist= 56
matched with= 6
index= 3
				this is wrong
pearson= 80
cosine= 80
c_dist= 80
matched with= 9
index= 3
				this is wrong
pearson= 83
cosine= 83
c_dist= 83
matched with= 9
index= 4
				this is wrong
pearson= 26
cosine= 26
c_dist= 26
matched with= 3
index= 4
				this is wrong
pearson= 87
cosine= 87
c_dist= 87
matched with= 9
index= 4
				this is wrong
1
Total Test Case: 12
Average Accuracy: 0.9345238095238096
Recognition Rate: 0.08333333333333333
